In [1]:
import random
import time

In [2]:
class Grid: 
    """
    Handle the grid of the tic-tac-toe only
    """
    
    def __init__(self, display):
        
        self.grid = [ 3 * [0] for _ in range(3) ]
        self.display = display
        
    
    def mark(self, marker, i, j):
        self.grid[i][j] = marker
    
    
    def is_empty(self, i, j):
        return self.grid[i][j] == 0
    
    
    def empty_cells(self):
        return [ (i, j) for i in range(3) for j in range(3) if self.is_empty(i, j) ]
    
    
    def _flatten(self):
        return [ x for row in self.grid for x in row ]
    
        
    def score_three_in_a_row(self):
        return [ x for sums in [ self.row_sum(), self.col_sum(), self.diag_sum() ] for x in sums ]
        
        
    def row_sum(self):
        return [ sum([ self.grid[i][j] for j in range(3)]) for i in range(3)]
    
    
    def col_sum(self):
        return [ sum([ self.grid[i][j] for i in range(3)]) for j in range(3)]
    
    
    def diag_sum(self):
        diags_idx = [ 
            [ (0, 0), (1, 1), (2, 2) ],
            [ (2, 0), (1, 1), (0, 2) ]
        ]
        return [ sum([ self.grid[i][j] for i, j in diag_idx ]) for diag_idx in diags_idx]
        
        
    def __str__(self):
        
        grid_displayed = [ self.display[marker] for marker in self._flatten() ]
        
        return """
            |i\\j| 0 | 1 | 2 |
            |---------------|
            | 0 | {} | {} | {} |
            |---------------|
            | 1 | {} | {} | {} |
            |---------------|
            | 2 | {} | {} | {} |
            |---------------|
        """.format(*grid_displayed)

In [3]:
class AbstractStrategy:
    
    def next_move(self):
        raise NotImplementedError("TicTacToe strategies must have next_move method implemented.")

In [12]:
class OneDepthStrategy(AbstractStrategy):
    
    def __init__(self, marker, grid):
        
        self.marker = marker
        self.grid = grid
        
        
    def winning_move(self, marker):
        
        row_sum = self.grid.row_sum()
        col_sum = self.grid.col_sum()
        diag_sum = self.grid.diag_sum()
        
        almost_win_score = 2*marker
        
        # Winning move in a row
        if almost_win_score in row_sum:
            i = row_sum.index(almost_win_score)
            j = [ j_ for j_ in range(3) if self.grid.is_empty(i, j_) ][0]
            return i, j
        
        # Winning move in a column
        if almost_win_score in col_sum:
            j = col_sum.index(almost_win_score)
            i = [ i_ for i_ in range(3) if self.grid.is_empty(i_, j) ][0]
            return i, j
        
        # Winning move in a diagonal
        if almost_win_score == diag_sum[0]: # Normal diagonal is winning
            i = [ i_ for i_ in range(3) if self.grid.is_empty(i_, i_)][0]
            return i, i
        if almost_win_score == diag_sum[1]: # Flipped diagonal is winning
            i = [ i_ for i_ in range(3) if self.grid.is_empty(i_, 2-i_) ][0]
            return i, 2-i    
        
        
        # No winning move
        return None
            
        
    def __call__(self):
        
        own_winning_move = self.winning_move(self.marker)
        opponent_winning_move = self.winning_move(-self.marker)
        
        if own_winning_move: # There's a possibility to win!
            return own_winning_move
        elif opponent_winning_move: # There's a possibility to prevent loss on the next move
            return opponent_winning_move
        else: # Choose a random valid move
            possible_moves = self.grid.empty_cells()
            random.shuffle(possible_moves)
            return possible_moves[0] 
        
        
    
    

In [13]:
class TicTacToe:
    """
    Handle the game logic
    
    Input:
        - start: str, choose between 'player', 'computer', 'random'
        - 
    
    TODO:
        Handle draw
        x Winning move detection
        Check if inputs are valid
    """
    
    def __init__(self, start='player', computer_strategy=OneDepthStrategy):
        
        self.display = {1:'X', -1:'O', 0:'.'}
        self.grid = Grid(display = self.display)
        
        # Decide the first player and define the next_player generator accordingly
        if start == 'random':
            start = random.choice(['player', 'computer'])
            
        if start == 'player':
            computer_move = computer_strategy(-1, self.grid)
            self.player = self._players(self._player_move, computer_move)
        elif start == 'computer':
            computer_move = computer_strategy(1, self.grid)
            self.player = self._players(computer_move, self._player_move)
        
        self.round = 1
        
    
    # Generator for player1 to player2 to player1 and so on
    def _players(self, player1_move, player2_move):
        
        while True:
            yield ('X', 1, player1_move)
            yield ('O', -1, player2_move)
    
    
    def _player_move(self):
        
        valid_move = False
        
        while not valid_move:
            time.sleep(.5)
            i, j = map(int, input("Give your move i j: ").split())
        
            if 0 <= i <= 2 and 0 <= j <= 2 and self.grid.is_empty(i, j):
                valid_move = True
            else:
                print(f"Your move at ({i}, {j}) is illegal. Please try again.")
                
        return i, j
        
        
    def outcome(self):
        
        sums = [ self.grid.row_sum(), self.grid.col_sum(), self.grid.diag_sum()]
        sums = [ x for sum_ in sums for x in sum_ ]
        
        winner = None
    
        for x in sums:
            if abs(x/3) == 1.:
                winner = int(x/3)
                break
                
        # No winner, but is it a draw?
        if self.grid.empty_cells(): # There is at least one empty cell left
            return winner
        else: # No empty cell, it's a draw
            return 'Draw'
    
    
    def play(self):
        
        outcome = None
        
        while outcome is None:
            
            current_marker, current_score, get_next_move = next(self.player)
        
            print(f"\n\n***** Round {self.round} | Player {current_marker} *****\n")
            print(self.grid)

            i, j = get_next_move()
            
            print(f"Playing {current_marker} at {i}, {j}")
            
            self.grid.mark(current_score, i, j)

            # Update for next loop
            self.round += 1
            outcome = self.outcome()

            
        if outcome == 'Draw': 
            print("\nThis is a draw...\n")
            print(self.grid)
        elif outcome: # There's a winner! (the player)
            print(f"\nWinner is {self.display.get(outcome, '?')}! Congrats!\n")
            print(self.grid)
        

In [16]:
if __name__ == '__main__':
    game = TicTacToe(start='random')
    game.play()



***** Round 1 | Player X *****


            |i\j| 0 | 1 | 2 |
            |---------------|
            | 0 | . | . | . |
            |---------------|
            | 1 | . | . | . |
            |---------------|
            | 2 | . | . | . |
            |---------------|
        
Give your move i j: 1 1
Playing X at 1, 1


***** Round 2 | Player O *****


            |i\j| 0 | 1 | 2 |
            |---------------|
            | 0 | . | . | . |
            |---------------|
            | 1 | . | X | . |
            |---------------|
            | 2 | . | . | . |
            |---------------|
        
Playing O at 2, 0


***** Round 3 | Player X *****


            |i\j| 0 | 1 | 2 |
            |---------------|
            | 0 | . | . | . |
            |---------------|
            | 1 | . | X | . |
            |---------------|
            | 2 | O | . | . |
            |---------------|
        
Give your move i j: 0 0
Playing X at 0, 0


***** Round 4 | Player O *****


           